# Extract All Encoder Attention Maps (Google Colab)

Extract encoder self-attention maps for all 2000 sentence pairs in both directions.

**Runtime:** ~1-2 hours on Colab GPU (vs ~5 hours on CPU)

**Before running:**
1. Upload your project folder to Google Drive
2. Set the `ROOT_DIR` variable below to point to your project directory
3. Enable GPU: Runtime → Change runtime type → GPU

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Configuration

**IMPORTANT:** Change `ROOT_DIR` to match your Google Drive folder structure

In [ ]:
# ============================================================================
# CHANGE THIS TO YOUR GOOGLE DRIVE PATH
# ============================================================================
ROOT_DIR = "/content/drive/MyDrive/UofT/CSC2517/term_paper"  # <-- CHANGE THIS

# Example paths:
# ROOT_DIR = "/content/drive/MyDrive/CSC2517/term_paper"
# ROOT_DIR = "/content/drive/MyDrive/research/term_paper"
# ============================================================================

import os
os.chdir(ROOT_DIR)
print(f"Working directory: {os.getcwd()}")

# Verify directories exist
assert os.path.exists("models/nllb-600M"), "Model directory not found!"
assert os.path.exists("data/sentence_pairs_en_zh.pkl"), "Data file not found!"
print("✓ All required files found")

## 3. Install and Import Dependencies

In [ ]:
# Install required packages with specific transformers version to avoid bugs
!pip install -q transformers==4.44.2 datasets torch pandas numpy tqdm

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from tqdm import tqdm
import time

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Check transformers version
import transformers
print(f"Transformers version: {transformers.__version__}")

## 4. Define Functions

In [ ]:
def extract_encoder_attention(text, src_lang, tgt_lang, tokenizer, model, device):
    """
    Extract encoder self-attention for a given source text.

    Args:
        text: Source text string
        src_lang: Source language code (e.g., 'eng_Latn', 'zho_Hans')
        tgt_lang: Target language code (e.g., 'zho_Hans', 'eng_Latn')
        tokenizer: NLLB tokenizer
        model: NLLB model
        device: torch device

    Returns:
        dict with keys:
            - tokens: List of source tokens
            - encoder_attention: Encoder self-attention (num_layers, num_heads, seq_len, seq_len)
            - translation: Generated translation text
    """
    # Set source language
    tokenizer.src_lang = src_lang

    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt").to(device)

    # Get target language BOS token
    tgt_lang_id = tokenizer.convert_tokens_to_ids(tgt_lang)

    # Generate translation with attention output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=tgt_lang_id,
            output_attentions=True,
            return_dict_in_generate=True,
            max_length=128
        )

    # Extract encoder attention (available in encoder_attentions)
    # Shape: (num_layers, batch_size, num_heads, seq_len, seq_len)
    encoder_attention = outputs.encoder_attentions
    encoder_attention = torch.stack([layer.squeeze(0) for layer in encoder_attention])  # (num_layers, num_heads, seq_len, seq_len)

    # Decode tokens
    input_tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0].cpu())
    translation = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    return {
        'tokens': input_tokens,
        'encoder_attention': encoder_attention.cpu().numpy(),
        'translation': translation
    }


def save_checkpoint(results, output_path, checkpoint_num):
    """Save checkpoint to avoid losing progress."""
    checkpoint_path = output_path.parent / f"{output_path.stem}_checkpoint_{checkpoint_num}.pkl"
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(results, f)
    print(f"  💾 Checkpoint saved: {checkpoint_path.name}")


print("✓ Functions defined")

## 5. Setup and Configuration

In [ ]:
print("="*80)
print("Extracting Encoder Attention Maps for All 2000 Sentence Pairs")
print("="*80)

# Configuration
CHECKPOINT_INTERVAL = 100  # Save checkpoint every N examples
OUTPUT_DIR = Path("data/attention_maps_en_zh")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_FILE = OUTPUT_DIR / "all_encoder_attention.pkl"

# Device setup
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("✓ Using CUDA (GPU)")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✓ Using MPS (Apple Silicon)")
else:
    device = torch.device("cpu")
    print("⚠️  Using CPU (this will be slow!)")
print()

## 6. Load Model and Data

In [ ]:
# Load model and tokenizer
model_path = "models/nllb-600M"
print(f"Loading model from {model_path}...")

# Try loading tokenizer with use_fast=False to avoid tokenizer.json corruption issues
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
    print("✓ Loaded tokenizer (slow tokenizer)")
except Exception as e:
    print(f"⚠️  Failed to load local tokenizer: {e}")
    print("Downloading tokenizer from HuggingFace instead...")
    tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_fast=False)
    print("✓ Downloaded tokenizer from HuggingFace")

# Try loading model
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_path,
        attn_implementation="eager"  # Required for extracting attention weights
    ).to(device)
    print("✓ Loaded model from local files")
except Exception as e:
    print(f"⚠️  Failed to load local model: {e}")
    print("Downloading model from HuggingFace instead (this may take a few minutes)...")
    model = AutoModelForSeq2SeqLM.from_pretrained(
        "facebook/nllb-200-distilled-600M",
        attn_implementation="eager"
    ).to(device)
    print("✓ Downloaded model from HuggingFace")

model.eval()
print("✓ Model ready!")
print()

In [ ]:
# Load data
data_path = Path("data/sentence_pairs_en_zh.pkl")
print(f"Loading data from {data_path}...")
data = pd.read_pickle(data_path)
df = pd.DataFrame(data)
df = df.rename(columns={'english': 'en', 'chinese': 'zh'})
print(f"✓ Loaded {len(df)} sentence pairs")
print()

## 7. Check for Existing Checkpoint

In [ ]:
# Check for existing checkpoint to resume from
start_idx = 0
results = []
checkpoint_files = sorted(OUTPUT_DIR.glob(f"{OUTPUT_FILE.stem}_checkpoint_*.pkl"))

if checkpoint_files:
    latest_checkpoint = checkpoint_files[-1]
    print(f"Found checkpoint: {latest_checkpoint.name}")
    with open(latest_checkpoint, 'rb') as f:
        results = pickle.load(f)
    start_idx = len(results)
    print(f"✓ Resuming from example {start_idx}")
    print()
else:
    print("No checkpoint found. Starting from the beginning.")
    print()

## 8. Extract Attention Maps

**This will take ~1-2 hours on GPU**

In [ ]:
print(f"Extracting attention maps for {len(df)} sentence pairs...")
print(f"Progress will be saved every {CHECKPOINT_INTERVAL} examples")
print()

start_time = time.time()

for idx in tqdm(range(start_idx, len(df)), desc="Processing", unit="pair"):
    en_text = df.iloc[idx]['en']
    zh_text = df.iloc[idx]['zh']

    try:
        # Extract English encoder attention (EN → ZH)
        en_result = extract_encoder_attention(
            text=en_text,
            src_lang='eng_Latn',
            tgt_lang='zho_Hans',
            tokenizer=tokenizer,
            model=model,
            device=device
        )

        # Extract Chinese encoder attention (ZH → EN)
        zh_result = extract_encoder_attention(
            text=zh_text,
            src_lang='zho_Hans',
            tgt_lang='eng_Latn',
            tokenizer=tokenizer,
            model=model,
            device=device
        )

        # Store results
        results.append({
            'idx': idx,
            'en_text': en_text,
            'zh_text': zh_text,
            'en_tokens': en_result['tokens'],
            'zh_tokens': zh_result['tokens'],
            'en_attention': en_result['encoder_attention'],
            'zh_attention': zh_result['encoder_attention'],
            'en_translation': en_result['translation'],
            'zh_translation': zh_result['translation']
        })

        # Save checkpoint periodically
        if (idx + 1) % CHECKPOINT_INTERVAL == 0:
            save_checkpoint(results, OUTPUT_FILE, idx + 1)

    except Exception as e:
        print(f"\n⚠️  Error processing pair {idx}: {e}")
        print(f"   EN: {en_text[:60]}...")
        print(f"   ZH: {zh_text[:60]}...")
        continue

elapsed_time = time.time() - start_time

print()
print("="*80)
print(f"✓ Extraction complete! Processed {len(results)} sentence pairs")
print(f"⏱️  Total time: {elapsed_time / 60:.1f} minutes ({elapsed_time / len(results):.2f} sec/pair)")
print()

## 9. Save Final Results

In [ ]:
print(f"Saving final results to {OUTPUT_FILE}...")
with open(OUTPUT_FILE, 'wb') as f:
    pickle.dump(results, f)
print(f"✓ Saved to {OUTPUT_FILE}")

# Print summary statistics
file_size_mb = OUTPUT_FILE.stat().st_size / (1024 * 1024)
print()
print("="*80)
print("Summary Statistics")
print("="*80)
print(f"Total sentence pairs: {len(results)}")
print(f"Output file size: {file_size_mb:.1f} MB")
print(f"Average attention matrix shape:")
if results:
    sample = results[0]
    print(f"  English: {sample['en_attention'].shape}")
    print(f"  Chinese: {sample['zh_attention'].shape}")
print()

# Clean up checkpoint files
print("Cleaning up checkpoint files...")
for checkpoint_file in OUTPUT_DIR.glob(f"{OUTPUT_FILE.stem}_checkpoint_*.pkl"):
    checkpoint_file.unlink()
    print(f"  🗑️  Removed {checkpoint_file.name}")

print()
print("="*80)
print("✅ All done!")
print("="*80)

## 10. Download Results (Optional)

If you want to download the results file directly from Colab:

In [ ]:
# Uncomment to download the results file
# from google.colab import files
# files.download(str(OUTPUT_FILE))